# 0. 추천시스템의 개요

1. 정의 : 사용자에게 상품을 제안하는 소프트웨어 도구 이자 기술
2. 목표 : 어떤 사용자에게 어떤 상품을 `어떻게 추천`할지에 대해 이해

**파레토의 법칙** <br>
상위 20%가 80%의 가치를 창출한다.

<br>

**롱테일의 법칙** <br>
하위 80%가 상위 20%의 가치보다 크다. <br>
인터넷의 발전으로 많은 사람이 접근 가능해지면서 롱테일의 법칙을 추구

![recommendation system history](./img/history.png)

# 1. 연관분석(Association Analysis)

**정의** <br>
룰기반의 모델로서 상품과 상품사이에 어떤 `연관`이 있는지 찾아내는 알고리즘

<br>

**연관의 정의?** <br>
첫번째, `얼마나(frequent)` 같이 구매가 되는가? <br>
두번째, A아이템을 구매하는 사람이 B아이템을 구매하는가? <br>
라는 규칙을 찾아내는 형태 

<br>

`장바구니 분석`이라고 표현하기도 함

**규칙평가지표** <br>
[관련 설명 블로그](https://needjarvis.tistory.com/59)
- **support(지지도)** : 추천할 만한 데이터 분석인지를 판단하기 위해 사용 <br>
$$
For \,the \, rule \, A \rightarrow B, support(A) = P(A)
$$

<br>

- **confidence(신뢰도)** : 상품A를 구매했을 때 상품B까지 이어질 확률 <br>
$$
confidence(A \rightarrow B) = {P(A, B) \over P(A)}
$$

<br>

- **lift(향상도)** : 두 사건이 동시에 얼마나 발생하는지 비율, 독립성 측정 <br>
$$
lift(A \rightarrow B) = {P(A, B) \over P(A) \, P(B)} 
$$

<br>

연관분석은 가능한 모든 경우의 수를 탐색해 지지도, 신뢰도, 향상도가 높은 규칙을 찾아내는 방식<br>
문제점 : 아이템의 증가에 따른 규칙의 수가 기하급수적으로 증가

# 2. Apriori 알고리즘

"빈번한 아이템셋은 하위 아이템셋 또한 빈번할 것이다"<br>
즉, "빈번하지 않은 아이템셋은 하위 아이템셋 또한 빈번하지 않다"는 개념을 이용해 아이템셋의 증가를 줄이는 방법

<br>

**아이디어**<br>
{2, 3}의 지지도 > {0, 2, 3}, {1, 2, 3}의 지지도<br>
P(2, 3) > P(0, 2, 3)

**알고리즘 순서**

1. k개의 item을 가지고 단일항목집단 생성
2. 단일항목집단에서 최소 지지도(support) [사용자가 설정하는 하이퍼파라미터] 이상의 항목만 선택
3. 2에서 선택된 항목만을 대상으로 2개항목집단 생성
4. 2개항목집단에서 최소 지지도 혹은 신뢰도 이상의 항목만 선택
5. 위의 과정을 k개의 k-item frequent set을 생성할 때까지 반복

<br>

**알고리즘의 장단점**
- 장점
    - 원리가 간단해 사용자가 쉽게 이해하고 의미를 파악할 수 있음
    - 유의미한 연관성을 갖는 구매패턴을 찾아줌
    
    <br>
    
- 단점
    - 데이터가 클 경우에 속도가 느리고 연산량이 많음
    - 실제 사용시에 많은 연관상품이 나타남
    - 상관관계에 대한 설명은 가능하나 인과관계는 설명하지 못함

## 2-1. Apriori 실습

In [ ]:
# pip install mlxtend

In [7]:
import mlxtend
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [8]:
data = np.array([
    ['우유', '기저귀', '주스'],
    ['양상추', '기저귀', '맥주'],
    ['우유', '양상추', '기저귀', '맥주'],
    ['양상추', '맥주']
])

In [9]:
# implicit feedback(희소 행렬)으로 만들어주는 과정
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns = te.columns_)
df

,기저귀,맥주,양상추,우유,주스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,True,True,True,False
3,False,True,True,False,False


In [12]:
%%time

from mlxtend.frequent_patterns import apriori
apriori(df, min_support = 0.5, use_colnames = True)

CPU times: user 17.7 ms, sys: 15.3 ms, total: 32.9 ms
Wall time: 33.9 ms


,support,itemsets
0,0.75,(기저귀)
1,0.75,(맥주)
2,0.75,(양상추)
3,0.50,(우유)
4,0.50,"(맥주, 기저귀)"
5,0.50,"(기저귀, 양상추)"
6,0.50,"(우유, 기저귀)"
7,0.75,"(맥주, 양상추)"
8,0.50,"(맥주, 기저귀, 양상추)"


# 3. FP-Growth 알고리즘

Apriori의 속도측면의 단점을 개선한 알고리즘

<br>

**알고리즘 순서**
1. 모든 거래를 확인해, 각 아이템마다 지지도 계산하고 최소 지지도 이상의 아이템만 선택
2. 모든 거래에서 빈도가 높은 아이템 순서대로 정렬
3. 부모 노드를 중심으로 한 거래를 자식노드로 추가해주면서 tree 생성
4. 새로운 아이템이 나올 경우에는 부모노드부터 시작하고, 그렇지 않으면 기존의 노드에서 확장
5. 위 과정을 모든 거래에 대해 반복해 FP Tree 만들고, 최소 지지도 이상의 패턴만 추출

<br>

**알고리즘 장단점**
- 장점
    - Apriori 알고리즘보다 빠르고 2번의 탐색만 필요로 함
    - 후보 itemset을 생성할 필요없음
    
<br>

- 단점
    - 대용량의 데이터셋에서 메모리를 효율적으로 사용하지 않음
    - Apriori 알고리즘에 비해 설계하기 어려움
    - 지지도의 계산이 FP Tree가 만들어지고 나서 가능함

## 3-1. FP-Growth 알고리즘 실습

In [13]:
%%time

from mlxtend.frequent_patterns import fpgrowth
fpgrowth(df, min_support = 0.5, use_colnames = True)

CPU times: user 5.52 ms, sys: 2.71 ms, total: 8.24 ms
Wall time: 15.3 ms


,support,itemsets
0,0.75,(기저귀)
1,0.50,(우유)
2,0.75,(양상추)
3,0.75,(맥주)
4,0.50,"(맥주, 기저귀)"
5,0.50,"(기저귀, 양상추)"
6,0.50,"(맥주, 기저귀, 양상추)"
7,0.50,"(우유, 기저귀)"
8,0.75,"(맥주, 양상추)"


FP-Growth 알고리즘이 Apriori 알고리즘보다 빠른 것을 알 수 있음

In [19]:
from mlxtend.frequent_patterns import association_rules
association_rules(fpgrowth(df, min_support = 0.4, use_colnames = True),
                  metric = 'confidence', min_threshold = 0.5, support_only = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(맥주),(기저귀),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
1,(기저귀),(맥주),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
2,(기저귀),(양상추),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
3,(양상추),(기저귀),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
4,"(맥주, 기저귀)",(양상추),0.50,0.75,0.50,1.000000,1.333333,0.1250,inf
5,"(맥주, 양상추)",(기저귀),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
6,"(기저귀, 양상추)",(맥주),0.50,0.75,0.50,1.000000,1.333333,0.1250,inf
7,(맥주),"(기저귀, 양상추)",0.75,0.50,0.50,0.666667,1.333333,0.1250,1.50
8,(기저귀),"(맥주, 양상추)",0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
9,(양상추),"(맥주, 기저귀)",0.75,0.50,0.50,0.666667,1.333333,0.1250,1.50
